# Analytics Lab Solution

#### Activity 1: Digital media schemes for the city library

We need to perform an analysis over time, similar to the "Digital vs Physical by Year" report, but we want to compare the various licensing managers for digital media.

Essentially, we want to count by year and `CheckoutType` records where
* UsageClass is Digital
* Year is prior to 2020

We want to keep as many records as we can which meet those criteria, and write a reasonably efficient query from the original CSV data.

In [ ]:
import coiled
from dask.distributed import Client

cluster = coiled.Cluster(name="training-cluster")
client = Client(cluster)
client

In [ ]:
import dask.dataframe as ddf

raw = ddf.read_csv('s3://coiled-training/data/checkouts-small.csv', storage_options={"anon": True})

raw

In [ ]:
refined = raw.query('UsageClass == "Digital" & CheckoutYear < 2020')

refined[['CheckoutYear', 'CheckoutType', 'Checkouts']] \
    .groupby(['CheckoutYear', 'CheckoutType']).agg({'Checkouts' : 'sum'}).compute()

#### Activity 2: Publishers

What are the top 50 publishers in the Seattle library system by...
* checkout activity (easier)
* library material holdings (harder)

Hints:
* Try to use Dask's `nlargest` or `nsmallest` for ordered results with a limit (like 50).
    * That approach is vastly more efficient than trying to sort a big dataset.
* For top publishers by library holdings...
    * the same item may appear in many months of data
    * Pandas/Dask doesn't have the same "COUNT DISTINCT" operator as SQL so you may have to get a bit creative
    * if you don't narrow (hint!) down the data, it will be hard to run this query with the allocated cluster resources

In [ ]:
q1 = raw.groupby('Publisher').agg({'Checkouts':'sum'})
q1

In [ ]:
q1.nlargest(50, 'Checkouts').compute()

In [ ]:
q2 = raw[['Publisher', 'Title']].drop_duplicates().groupby('Publisher').agg({'Publisher':'count'})
q2

In [ ]:
q2.nlargest(50, 'Publisher').compute()

#### Activity 3: Popular subjects

*Bonus Project*

Notice that the Subjects field contains a string list of subjects.

If we want to analyze checkouts by subject, we might start by trying to parse this field into a Python list. Like Pandas, Dask allows us to split strings as well as explode collections into multiple rows.

Try to find the top 10 subjects by checkout activity. Hint: Try to eliminate as much data as you can from the dataset as early as possible.

In [ ]:
narrowed = raw[['Checkouts', 'Subjects']]
narrowed.head()

In [ ]:
narrowed.Subjects.str.split(', ').head()

In [ ]:
narrowed['Subject'] = narrowed.Subjects.str.split(', ')
narrowed = narrowed.drop(columns='Subjects')
narrowed.head()

In [ ]:
narrowed.explode('Subject').head()

We can check that the basic logic works...

In [ ]:
narrowed.explode('Subject').groupby('Subject').agg({'Checkouts':'sum'}).head()

But for large datasets you might want to narrow down the volume of data earlier -- i.e., the query above runs on the full amount of data and the `head` call can only limits the rows that come back from the result because the limit can't be pushed through the aggregation. 

Another way to "test" the query is to pull, say, two partitions:

In [ ]:
narrowed.partitions[:2].explode('Subject').groupby('Subject').agg({'Checkouts':'sum'}).head()

In [ ]:
narrowed.explode('Subject').groupby('Subject').agg({'Checkouts':'sum'}).nlargest(10, 'Checkouts').compute()